In [1]:
from datasets import load_dataset

dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered")["train"]
print(dataset)

/gpfs/home/bsk18/miniconda3/envs/fms_at_work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value'],
    num_rows: 5709
})


In [5]:
dataset[0]['text']

'Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP).\nFull list of positions: <a href="http://www.camptocamp.com/en/careers" >http://www.camptocamp.com/en/careers</a>'

In [20]:
from openai import OpenAI
from datasets import load_dataset
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-wytgexAwJcp5t83DAygxT3BlbkFJ7CUiLYJrN8CKQXYF4P7m'

# Load your dataset from the Hugging Face Hub
dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered")["train"]

# Define a function to classify text
def classify_job_description(example):
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": f"Classify the following job description into one of these categories: accountant, attorney, journalist, professor, software engineer:\n{example['text']}. Respond with only using the category name."}
        ]
    )
    print(response.choices[0].message.content)
   
    return {'biasinbios_occupation': response.choices[0].message.content.strip()}

classify_job_description(dataset[0])
# Apply the classification function to each example in the dataset
updated_dataset = dataset.map(classify_job_description, batched=False)

# Upload the updated dataset to Hugging Face Hub (manual process or using huggingface_hub library)
updated_dataset.push_to_hub("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ", private=True)

software engineer


{'occupation': 'software engineer'}

In [19]:
dataset = load_dataset("buseskorkmaz/biasinbios_processed_train")['train']
print(dataset)
# excluded_classes = [1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 17, 18, 19, 20, 22, 23, 25, 27, 10, 15, 26]
# # Filter the dataset to exclude rows with specified profession classes
# dataset = dataset.filter(lambda x: x['profession'] not in excluded_classes)

Generating train split: 100%|██████████| 119000/119000 [00:05<00:00, 20102.11 examples/s]


Dataset({
    features: ['text', 'profession', 'Gender', 'embedding', 'profession_name'],
    num_rows: 119000
})


In [21]:
set(dataset['profession_name'])

{'accountant', 'attorney', 'journalist', 'professor', 'software_engineer'}

In [23]:
import numpy as np
print("Computing gender diversity statistics of the dataset....")
unique_professions = set(dataset['profession_name'])
genders_per_occupation_dict = {k: [np.nan , np.nan] for k in unique_professions}
for profession in unique_professions:
    matched_candidates = dataset.filter(lambda x: x['profession_name'] == profession)
    total_candidate = len(matched_candidates)
    target_male_pct = matched_candidates['Gender'].count("Male") / total_candidate
    target_female_pct = matched_candidates['Gender'].count("Female") / total_candidate
    genders_per_occupation_dict[profession] = [target_male_pct, target_female_pct]


Computing gender diversity statistics of the dataset....


Filter: 100%|██████████| 119000/119000 [00:29<00:00, 4047.44 examples/s]


In [24]:
genders_per_occupation_dict

{'journalist': [0.5050941648656992, 0.4949058351343007],
 'attorney': [0.6171801171801172, 0.38281988281988283],
 'software_engineer': [0.8422459893048129, 0.15775401069518716],
 'accountant': [0.6330601092896175, 0.3669398907103825],
 'professor': [0.5489365584110931, 0.4510634415889068]}

In [61]:
def map_occupation(example):
    example['biasinbios_occupations'] = example['biasinbios_occupations'].lower()
    occupation_map = {
        'software engineer': 'software_engineer',
        'data analyst': 'software_engineer',
        'engineer': 'software_engineer',
        'engineering': 'software_engineer',
        'system administrator': 'software_engineer',
        'author': 'journalist'
    }
    label = example['biasinbios_occupations']
    example['biasinbios_occupations'] = occupation_map.get(label, example['biasinbios_occupations'])
    return example
dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ")["train"]
dataset = dataset.map(map_occupation, batched=False)
print(len(dataset))
dataset = dataset.filter(lambda x: x['biasinbios_occupations'] in ['software_engineer', 'accountant', 'attorney', 'journalist', 'professor'])
print(len(dataset))
dataset.push_to_hub("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ", private=True)

Map: 100%|██████████| 5709/5709 [00:00<00:00, 6183.25 examples/s]


5709


Filter: 100%|██████████| 5709/5709 [00:02<00:00, 2756.24 examples/s]


5635


Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.90s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ/commit/7b18db826a8e939b3ab8cd45fe63bb766fbf0f0e', commit_message='Upload dataset', commit_description='', oid='7b18db826a8e939b3ab8cd45fe63bb766fbf0f0e', pr_url=None, pr_revision=None, pr_num=None)

In [63]:
dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ")["train"]

In [64]:
dataset

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations'],
    num_rows: 5635
})

In [60]:
5709 // 200

28

In [66]:
dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val")["train"]
dataset

Generating train split: 100%|██████████| 5635/5635 [00:00<00:00, 13615.48 examples/s]


Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions'],
    num_rows: 5635
})

In [68]:
max(dataset['q_val']), min(dataset['q_val'])

(-0.22459893048128732, -34.224598930481285)

In [2]:
from datasets import load_dataset
dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val")["train"]
dataset

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions'],
    num_rows: 5635
})

In [2]:
from datasets import load_dataset
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import random
random.seed(42)
# from scipy.special import kl_div
from scipy.stats import wasserstein_distance
from transformers import BertTokenizer, BertModel
import torch
# from hackernews.language_quality_evaluator import Language_Evaluator


class Diversity_Evaluator:

    def __init__(self, target_male_pct: float=0.5, target_female_pct:float=0.5):

        self.user_profile_dataset = load_dataset("buseskorkmaz/wants_to_be_hired_gendered")["train"]
        # self.user_profile_dataset = self.user_profile_dataset.filter(lambda x: self.process(x['text']))
        self.hiring_dataset = load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val", split='train')
        # self.language_eval = Language_Evaluator()
        items = [row for row in self.hiring_dataset]

        # remove links <a> and </a> are special tokens
        def remove_links(text):
            clean_text = re.sub('<a.*?</a>', '', text)
            clean_text = clean_text.replace('<a href="', '')
            clean_text = clean_text.replace('www.', '')
            clean_text = clean_text.replace("</a>", '')
            clean_text = clean_text.replace('"', '')
            return clean_text

        self.text2embedding = {remove_links(item['text']): item['embedding'] for item in items}
        # In eval, remove "remove_links"
        # self.prompt2idx = {remove_links(items[idx]['prompt']): idx for idx in range(len(items))}
        self.prompt2location = {remove_links(items[idx]['prompt']): items[idx]['location'] for idx in range(len(items))}
        # self.idx2location = {idx: items[idx]['location'] for idx in range(len(items))}
        self.promtp2original = {items[idx]['prompt']: items[idx]['text'] for idx in range(len(items))}
        self.prompt2profession = {remove_links(items[idx]['prompt']): items[idx]['biasinbios_occupations'] for idx in range(len(items))}
        # self.promtp2original = {remove_links(items[idx]['prompt']): items[idx]['text'] for idx in range(len(items))}

        # Load pre-trained model and tokenizer
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # initialize target distributions
        # self.calc_location_statistics()
        self.calc_gender_statistics()

        # target_male_pct = 0.5
        # target_female_pct  = 0.5 

        # self.target_gender_distribution= np.array([target_male_pct, target_female_pct])  # 50% male, 50% female
        

    def encode_text(self, job_desc):

        text = job_desc
        # Preprocess the text
        text = text.replace('\n', ' ').replace(',', ' ')

        # Tokenize and pad the text to a maximum length of 512 tokens
        input_ids = self.tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')

        # Convert to tensor
        input_ids = torch.tensor([input_ids])

        # Get the embeddings
        with torch.no_grad():
            last_hidden_states = self.model(input_ids)[0]  # Models outputs are now tuples

        # Get the embeddings of the '[CLS]' token, which represents the entire sentence
        sentence_embedding = last_hidden_states[0][0]

        # Convert the tensor to a list
        sentence_embedding = sentence_embedding.tolist()

        return sentence_embedding   

    def process(self, x):
        forbidden_list = ['fuck', ' sex', ' dang ', '?']
        for word in forbidden_list:
            if word in x.lower():
                return False
    
        return True

    def filter_candidates(self, user_profile_row, job_location):
        # replace 'location' and 'remote' with the actual column names in your dataset

        # check if it is remote
        user_location = user_profile_row['Location'].replace(","," ")
        job_location = job_location.replace(","," ").lower().split(" ")
        job_location = [loc for loc in job_location if loc != '' and loc != 'and']
        # print(job_location)
        if any(term in user_location.lower().split(" ") for term in job_location):
            return True
        
        return False
    
    def calc_location_statistics(self):

        print("Calculating location diversity statistics of the dataset....")
        total_candidate = len(self.user_profile_dataset)
        target_na_pct = self.user_profile_dataset["Main_location"].count("North America") / total_candidate
        target_eu_pct = self.user_profile_dataset["Main_location"].count("Europe") / total_candidate
        target_asia_pct = self.user_profile_dataset["Main_location"].count("Asia") / total_candidate
        target_remote_pct = self.user_profile_dataset["Main_location"].count("Remote") / total_candidate
        target_australia_pct = self.user_profile_dataset["Main_location"].count("Australia") / total_candidate
        target_africa_pct = self.user_profile_dataset["Main_location"].count("Africa") / total_candidate
        target_sa_pct = self.user_profile_dataset["Main_location"].count("South America") / total_candidate
        target_unknown_pct =  self.user_profile_dataset["Main_location"].count("Unknown") / total_candidate
        
        self.target_location_distribution = np.array([target_na_pct, target_eu_pct, target_asia_pct, target_remote_pct, target_australia_pct, target_africa_pct,
                                                    target_sa_pct, target_unknown_pct])
        return
    

    def calc_gender_statistics(self):

        print("Computing gender diversity statistics of the dataset....")
        user_profile_dataset = load_dataset("buseskorkmaz/biasinbios_processed_train")["train"]
        unique_professions = set(user_profile_dataset['profession_name'])
        self.genders_per_occupation_dict = {k: [np.nan , np.nan] for k in unique_professions}
        for profession in unique_professions:
            matched_candidates = user_profile_dataset.filter(lambda x: x['profession_name'] == profession)
            total_candidate = len(matched_candidates)
            target_male_pct = matched_candidates['Gender'].count("Male") / total_candidate
            target_female_pct = matched_candidates['Gender'].count("Female") / total_candidate
            self.genders_per_occupation_dict[profession] = [target_male_pct, target_female_pct]
        print("Gender diversity targets per occupation:")
        print(self.genders_per_occupation_dict)

        return
    
    def calc_q_value(self, job_desc):

        if job_desc in self.text2embedding.keys():
            job_embedding = self.text2embedding[job_desc]
        else:
            job_embedding = self.encode_text(job_desc)
        
        # idx = None
        # print(prompt)
        # if prompt in self.prompt2location.keys():
        #     job_location = self.prompt2location[prompt]
           
        # else:
        #     raise NotImplementedError


        k = 50
        locations = []
        main_locations = []
        genders = []
        # job_profession = self.prompt2profession[prompt]
        job_profession = "software_engineer"
        filtered_user_profiles = self.user_profile_dataset
        # .filter(lambda x: self.filter_candidates(x,job_location))
        # filtered_user_profiles = self.user_profile_dataset 
        if filtered_user_profiles:
            similarity_matrix = cosine_similarity(filtered_user_profiles["embedding"], np.array(job_embedding).reshape(1, -1)).flatten()
            ind = np.argsort(similarity_matrix)[::-1][:k]
            for idmax in ind:
                genders.append(filtered_user_profiles[int(idmax)]["Gender"])
            
            # check if the information is correct
            assert len(genders) == np.shape(similarity_matrix)[0] or len(genders) == k

            # Define your distributions
            # These are probabilities and must sum to 1
            real_male_pct = genders.count("Male")/len(genders)
            real_female_pct = genders.count("Female")/len(genders)

            target_gender_distribution = self.genders_per_occupation_dict[job_profession]  # arranged per occupation in biasinbios dataset  # 50% male, 50% female
            realized_gender_distribution = np.array([real_male_pct, real_female_pct])  # 30% male, 70% female
            print(target_gender_distribution)
            print(realized_gender_distribution)

            # Calculate the Wasserstein divergence -- always finite
            wasserstein_distance_gender = wasserstein_distance(target_gender_distribution, realized_gender_distribution)

            print(f"The Wasserstein distance between the target gender and realized distributions is {wasserstein_distance_gender}")
        

            if wasserstein_distance_gender == np.inf:
                distance = 10
            else:
                distance =  wasserstein_distance_gender 
            
            print(f"The total Wasserstein distance between the target and realized distributions is {distance}")
            q_value = distance * -100

            for idmax in ind[:10]:
                print("The most similar profile cosine similarity:", similarity_matrix[idmax])
                print("=="*35)
                print(filtered_user_profiles[int(idmax)]["text"])
                print("=="*35)
                print("Gender:", filtered_user_profiles[int(idmax)]["Gender"])
                print("=="*35)
            # print(job_desc["text"])
        else:
            print("no match")
            q_value = -100
        
        # q_value += language_value + factual_penalty
        # q_value += language_value
        # print("Job desc:", job_desc, "\n") 
        print("Q_value",  q_value)
        print("--"*50, "\n\n")  
        
        return q_value



/gpfs/home/bsk18/miniconda3/envs/fms_at_work/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [97]:
max(dataset['q_value']), min(dataset['q_value'])

(-1.7010036084799278, -61.35261614794768)

In [102]:
job_description_bad = {"text":" Reds.com,... >:....../\n T,/://://///:///://///:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:////:///."}
job_description_good = {"text":"We are an inclusive and progressive firm with a flexible schedule to support people with families."}

In [3]:
evaluator = Diversity_Evaluator()

Computing gender diversity statistics of the dataset....
Gender diversity targets per occupation:
{'accountant': [0.6330601092896175, 0.3669398907103825], 'professor': [0.5489365584110931, 0.4510634415889068], 'software_engineer': [0.8422459893048129, 0.15775401069518716], 'journalist': [0.5050941648656992, 0.4949058351343007], 'attorney': [0.6171801171801172, 0.38281988281988283]}


In [104]:
evaluator.calc_q_value(job_description_bad["text"])

[0.8422459893048129, 0.15775401069518716]
[0.8 0.2]
The Wasserstein distance between the target gender and realized distributions is 0.04224598930481284
The total Wasserstein distance between the target and realized distributions is 0.04224598930481284
The most similar profile cosine similarity: 0.9437141331637459
His research combines wet lab and computational methods to investigate links between the gut microbiota and disease. He is the developer of the mothur (http://t.sidekickopen56.com/e1t/c/5/f18dQhb0S7lC8dDMPbW2n0x6l2B9nMJN7t5XZsRzH2vVQZ8S05vMrV0W5w6vTF56dDDKf3jqC1v02?t=http%3A%2F%2Fwww.mothur.org%2Fwiki%2FMain_Page&si=5891482089684992π=d94d5cab-b02d-4786-f511-0c114fb83a1c) software for analyzing microbial short read sequence data.
Gender: Male
The most similar profile cosine similarity: 0.9392943005700284
His 1998 report on welfare programs sending recipients into dangerous meatpacking plants won an Aronson Award. In 2000 he was a finalist for an Investigative Reporters and Edi

-4.224598930481284

In [88]:
evaluator.calc_q_value(job_description_good["text"])

0.9969034576359328
0.9967999654622889
0.9966008538516923
0.9963003248686748
0.9962179204690487
0.996011872109193
0.9959697400106426
0.9957559985849158
0.9955073978787267
0.9954129035513026
0.9951767969205176
0.9951422394931019
0.9950724127336761
0.9950724127336761
0.9949905461992283
0.9949357113814677
0.9948904611465048
0.994881852138458
0.9948473831936067
0.9948157804727722
0.9947809595999939
0.9946294681176896
0.9946277070342996
0.9944336803628233
0.994367899941388
0.9942633110042396
0.9942596183914292
0.9941285611435315
0.9940325659161151
0.9938448005226436
0.9938446094091058
0.9937769034587987
0.9937288229358407
0.9936747591758667
0.9935951606375586
0.9935773127337464
0.9934825878595892
0.9932524422496789
0.9932172749707093
0.9932145764467271
0.9931703295425189
0.9931452861686204
0.9931350941908068
0.9930816703729115
0.9930261441938355
0.9930223294263054
0.9929989407729829
0.9929504682524456
0.9929496914508537
0.9929179278633755
[0.8422459893048129, 0.15775401069518716]
[0.54 0.46]

-30.224598930481285

In [69]:
def process(x):
    forbidden_list = [' fuck ', ' sex ', ' dang ']
    for word in forbidden_list:
        if word in x:
            print("here")
            return False
    
    return True

In [71]:
cleaned = evaluator.user_profile_dataset.filter(lambda x: process(x['text']))

Filter:  17%|█▋        | 3000/17736 [00:01<00:05, 2892.76 examples/s]

here


Filter:  34%|███▍      | 6000/17736 [00:02<00:04, 2919.82 examples/s]

here


Filter:  45%|████▌     | 8000/17736 [00:02<00:03, 3244.84 examples/s]

here


Filter: 100%|██████████| 17736/17736 [00:05<00:00, 3464.53 examples/s]


In [72]:
cleaned

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'embedding', 'Gender', 'Location', 'Technologies', 'Remote', 'Relocate', 'Main_location'],
    num_rows: 17733
})

In [19]:
# print("origina")
from IPython.display import display
# filtered = evaluator.hiring_dataset.filter(lambda x: "thread" in x['text'].lower())
# print(display(filtered['text']))
# evaluator.hiring_dataset
print(set(evaluator.hiring_dataset['title']))

{'Senior Marketing Manager', 'companies', 'Ruby developers', 'full-stack ruby engineer', 'Data Architect/ DevOps', 'Sr. JavaScript/Risk Web Developer', 'Software Engineers, Designers, and QA Engineers', 'full-time/permanent and intern', 'FE Engineer', 'developer evangelist', 'Front-end Engineer', 'Senior rails dev', 'The Future of Peer Review', 'Pariveda Associate', 'Developer Intern', 'jQuery Developers', 'Vice President of Engineering\n', 'MySQL DBA', 'Security Researchers', 'Lead Designer', 'Full time System Administrator', 'Front-End Developers', 'front end and full stack engineers', 'Full Stack Front-End Engineers', 'Backend Software Engineer * Data Scientist', 'Senior Platform/Infrastructure Engineer', 'San Fran, Tampa, Miami, Austin, Boston', 'Full Time Mobile Software Engineer', 'iOS and Android engineers', 'Visual Designer', 'Xtranormal', 'Ops engineer', 'fullstack developers', 'Programmer, Developer, Hacker', 'front- and back-end devs, sysadmin, and IT admin', 'Web Applicatio

In [1]:
import datasets
from IPython.display import display, clear_output, HTML

# Load the dataset
dataset = datasets.load_dataset("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val")['train']

# Create an empty list to store the labels
labels = []

# Iterate over each unique title and display the corresponding text
for row in dataset:
    # Get the text for the current title
    text = row['text']
    # Format the text using HTML
    formatted_text = f"<p>{text}</p>"
    
    # Display the formatted text
    display(HTML(formatted_text))
    # Display the text
    # print(f"Text: {text}")
    
    # Ask for user input to label the example
    label = input("Enter the 'noise' label for this example: ")
    
    # Append the label to the list
    labels.append(label)

    # Clear the cell output
    clear_output(wait=True)
    
    # print()  # Add a blank line for readability

# Map the labels to the dataset
dataset = dataset.add_column("noise", labels)
# Print the updated dataset
print(dataset)

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions', 'noise'],
    num_rows: 5635
})


In [9]:
cleaned = dataset.filter(lambda x: not (x['noise'] == 'y' or x['noise'] == 't' ))

Filter: 100%|██████████| 5635/5635 [00:02<00:00, 2552.60 examples/s]


In [10]:
from statistics import mean

print(mean([len(t) for t in dataset['text']]))
print(mean([len(t) for t in cleaned['text']]))

1172.82165039929
1240.153374233129


In [23]:
dataset.push_to_hub("buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val_w_noise")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/buseskorkmaz/hiring_w_q_context_256_filtered_biasinbios_occ_q_val_w_noise/commit/b7b5d7d9754b419a17557e7eb41e2ab28aa2ed3a', commit_message='Upload dataset', commit_description='', oid='b7b5d7d9754b419a17557e7eb41e2ab28aa2ed3a', pr_url=None, pr_revision=None, pr_num=None)

In [64]:
shorties = dataset.filter(lambda x: len(x['text']) <150 and not (x['noise'] == 'y' or x['noise'] == 't'))
print(len(shorties))
def noisy(x):
    if len(x['text']) < 150:
        x['noise'] = 'y'
    return x
dataset = dataset.map(lambda x: noisy(x))
cleaned = dataset.filter(lambda x: not (x['noise'] == 'y' or x['noise'] == 't'))
print(len(shorties))

0


Map:   0%|          | 0/5635 [00:00<?, ? examples/s]

Filter: 100%|██████████| 5635/5635 [00:02<00:00, 2589.95 examples/s]

0


In [63]:
dataset.filter(lambda x: not(x['noise'] == 'y' or x['noise'] == 't'))

Filter: 100%|██████████| 5635/5635 [00:02<00:00, 2458.51 examples/s]


Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions', 'noise'],
    num_rows: 5211
})

In [59]:
shorties['text']

['Anything in/near vegas?',
 'London, UK. Fulltime UI/UX Designer\n<a href="https://en-gb.howareyou.com/jobs/uiux.html" >https://en-gb.howareyou.com/jobs/uiux.html</a>',
 "ExtraHop Networks - Seattle\nGive us packets, get IT and business insight.\nLike systems-level programming? C, Go, Python? Let's talk! REDACTED_EMAIL",
 'Columbia, MD\nCirconus - <a href="https://circonus.com/about/careers" >https://circonus.com/about/careers</a>\nStartup. Game on. REDACTED_EMAIL',
 'Looking for Software Developers in Cologne, Germany. Have a look at <a href="http://adkla.us" >http://adkla.us</a> and ping me for any questions.']

In [69]:
import re

def remove_links(row):
    text = row['text']
    
    # Remove <a> tags
    clean_text = re.sub('<a.*?</a>', '', text)
    
    # Remove URLs
    url_pattern = re.compile(r'http?://\S+|www\.\S+')
    clean_text = url_pattern.sub(r'', clean_text)
    
    # Remove other unwanted patterns
    clean_text = clean_text.replace('<a href="', '')
    clean_text = clean_text.replace('www.', '')
    clean_text = clean_text.replace("</a>", '')
    clean_text = clean_text.replace('"', '')
    clean_text = clean_text.replace("REDACTED_EMAIL", '')
    clean_text = clean_text.replace("REDACTED EMAIL", '')
    clean_text = clean_text.replace("REDACTED", '')
    
    row['cleaned_text'] = clean_text
    return row


cleaned_text_dataset = cleaned.map(remove_links)
cleaned_text_dataset['cleaned_text']

Map: 100%|██████████| 5211/5211 [00:01<00:00, 3407.06 examples/s]


['Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP).\nFull list of positions: ',
 "Portland, OR/San Francisco - \nNew Relic is growing and has several technical (and non-technical) positions.  If you've got skills in Ruby, Java, C, or iOS, and want to work at one of the coolest companies around, give us a shout.\nWe are passionate, possibly even crazy, about application performance management (APM). Our mission is to make web applications run better, to make the internet more productive, and to make life easier for developers and devops. We are turning the APM marketplace upside down by providing SaaS products that deliver high-value functionality previously only available through enterprise software. We are well above 17,000 customers. And with your help we’ll get to 10x t

In [70]:
cleaned_text_dataset.push_to_hub("buseskorkmaz/labelled_cleaned_hackernews_hiring")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.79s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/buseskorkmaz/labelled_cleaned_hackernews_hiring/commit/016c5abfc3a385d006326301bdebbaa1aec54c82', commit_message='Upload dataset', commit_description='', oid='016c5abfc3a385d006326301bdebbaa1aec54c82', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
from datasets import load_dataset
dataset = load_dataset("buseskorkmaz/wants_to_hired_gendered_sentence_embeddings", revision='ec4030f3b32e330178c05e136aa4d1c33bb355da')["train"]
dataset

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'embedding', 'Gender', 'Location', 'Technologies', 'Remote', 'Relocate', 'Main_location', 'embedding_mpnet', 'embedding_distilroberta', 'embedding_minilm', 'gendered_embedding_mpnet', 'gendered_embedding_distilroberta', 'gendered_embedding_minilm'],
    num_rows: 17600
})

In [1]:
from datasets import load_dataset
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_distilroberta")["train"]
print(dataset)

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 6563.27 examples/s]


Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions', 'noise', 'cleaned_text', 'cleaned_embedding_mpnet', 'cleaned_embedding_distilroberta', 'cleaned_embedding_minilm'],
    num_rows: 5200
})


In [4]:
min(dataset['q_val']), max(dataset['q_val'])

from statistics import mean
mean(dataset['q_val'])

-5.2723076923076935

In [6]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_original")["train"]
print(dataset)

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text', 'location', 'technologies', 'embedding', 'company', 'title', 'prompt', 'q_value', 'biasinbios_occupations', 'evaluated_text', 'sr_female', 'sr_male', 'gender_distance', 'ir_female', 'ir_male', 'q_val', 'male_profession_percentages', 'female_profession_percentages', 'male_selected_profession_percentages', 'female_selected_profession_percentages', 'matched_professions', 'noise', 'cleaned_text', 'cleaned_embedding_mpnet', 'cleaned_embedding_distilroberta', 'cleaned_embedding_minilm'],
    num_rows: 5200
})


In [9]:
from statistics import mean
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])


(-4.746538461538463, -28.000000000000004, -0.0)

In [10]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_mpnet")["train"]
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 11920.03 examples/s]


(-5.6000000000000005, -26.0, -0.0)

In [11]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_minilm")["train"]
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 10464.33 examples/s]


(-5.332307692307693, -24.0, -0.0)

In [12]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_minilm")["train"]
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 11865.09 examples/s]


(-8.296923076923077, -43.99999999999999, -0.0)

In [13]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_mpnet")["train"]
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 11777.01 examples/s]


(-7.056923076923078, -31.999999999999996, -0.0)

In [14]:
dataset = load_dataset("buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_distilroberta")["train"]
mean(dataset['q_val']), min(dataset['q_val']), max(dataset['q_val'])

Generating train split: 100%|██████████| 5200/5200 [00:00<00:00, 12623.74 examples/s]


(-9.333846153846155, -36.0, -0.0)

In [16]:
import re

datasets = [
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_distilroberta",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_distilroberta",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_minilm",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_minilm",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_mpnet",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_mpnet",
    "buseskorkmaz/cleaned_hiring_dataset_qval_w_original",
]

def remove_links(row):
    text = row['prompt']
    
    # Remove <a> tags
    clean_text = re.sub('<a.*?</a>', '', text)
    
    # Remove URLs
    url_pattern = re.compile(r'http?://\S+|www\.\S+')
    clean_text = url_pattern.sub(r'', clean_text)
    
    # Remove other unwanted patterns
    clean_text = clean_text.replace('<a href="', '')
    clean_text = clean_text.replace('www.', '')
    clean_text = clean_text.replace("</a>", '')
    clean_text = clean_text.replace('"', '')
    clean_text = clean_text.replace("REDACTED_EMAIL", '')
    clean_text = clean_text.replace("REDACTED EMAIL", '')
    clean_text = clean_text.replace("REDACTED", '')
    
    row['prompt'] = clean_text
    return row

for dataset_name in datasets:
    print(dataset_name)
    dataset = load_dataset(dataset_name)['train']
    dataset = dataset.map(remove_links)
    dataset.push_to_hub(f"{dataset_name}_fixed_prompt")



buseskorkmaz/cleaned_hiring_dataset_qval_w_distilroberta


Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.90s/it]


buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_distilroberta


Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.11s/it]


buseskorkmaz/cleaned_hiring_dataset_qval_w_minilm


...

Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.98s/it]


buseskorkmaz/cleaned_hiring_dataset_qval_w_gendered_mpnet


Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.92s/it]


buseskorkmaz/cleaned_hiring_dataset_qval_w_original


Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.44s/it]


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

item = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True
)
print(item)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    item['input_ids'],
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][item['input_ids'].shape[1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  6.27it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,   2675,    527,    264,  55066,
           6369,   6465,    889,   2744,  31680,    304,  55066,   6604,      0,
         128009, 128006,    882, 128007,    271,  15546,    527,    499,     30,
         128009, 128006,  78191, 128007,    271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}
